In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip -P Skoltech '/content/drive/My Drive/hack_data.zip'

Archive:  /content/drive/My Drive/hack_data.zip
  inflating: clients.csv             
  inflating: materials.csv           
  inflating: plants.csv              
  inflating: transactions.parquet    


In [ ]:
!python -m pip install "dask[complete]" 

     |████████████████████████████████| 92kB 2.4MB/s 
     |████████████████████████████████| 655kB 9.0MB/s 
     |████████████████████████████████| 102kB 7.6MB/s 
  Created wheel for locket: filename=locket-0.2.0-cp36-none-any.whl size=4040 sha256=fe0bf940b595e25ecdb817155ce62cab9336f3270c5506ff95a9157137b46c4d
  Stored in directory: /root/.cache/pip/wheels/26/1e/e8/4fa236ec931b1a0cdd61578e20d4934d7bf188858723b84698
  Created wheel for contextvars: filename=contextvars-2.4-cp36-none-any.whl size=7666 sha256=38e9f59f88faef79845ae9fd5943e9943c3a9afe9505bcbe494aaf5f3ecfa0bf
  Stored in directory: /root/.cache/pip/wheels/a5/7d/68/1ebae2668bda2228686e3c1cf16f2c2384cea6e9334ad5f6de
Successfully built locket contextvars
ERROR: distributed 2.27.0 has requirement cloudpickle>=1.5.0, but you'll have cloudpickle 1.3.0 which is incompatible.
  Found existing installation: distributed 1.25.3
    Uninstalling distributed-1.25.3:
      Successfully uninstalled distributed-1.25.3


In [ ]:
import dask.dataframe as dd
import pandas as pd
import pickle
from tqdm import tqdm

In [ ]:
clients = pd.read_csv('clients.csv', verbose=True)
materials = pd.read_csv('materials.csv', verbose=True)
plants = pd.read_csv('plants.csv', verbose=True)

Tokenization took: 32.42 ms
Type conversion took: 49.64 ms
Parser memory cleanup took: 0.01 ms
Tokenization took: 61.90 ms
Type conversion took: 67.01 ms
Parser memory cleanup took: 0.01 ms
Tokenization took: 27.93 ms
Type conversion took: 39.52 ms
Parser memory cleanup took: 0.01 ms
Tokenization took: 0.08 ms
Type conversion took: 0.80 ms
Parser memory cleanup took: 0.00 ms


In [ ]:
trans = dd.read_parquet('transactions.parquet')

In [ ]:
trans_by_client = trans.groupby('client_id')

NameError: ignored

In [ ]:
print(trans_by_client.last().head())

                                                            chq_id  ... is_promo
client_id                                                           ...         
0000d48a8891cd462fdf01ea861ec53e  5e58f7ad4e48291073ee4807d952dd62  ...        1
00028a1dcb30863404a386961a69300e  d82790f079a00571d4b0f0fcda3153e7  ...        1
0004274683494a9eca5e70ef97299d55  375e0d72b26f5cd483324cc11f6d0f58  ...        0
00072f81efcb3481861f7300c5e2a88a  fce88284d1aacdc26f98434ee4975ef4  ...        0
00076596eb749cf10c2e984621dffe95  2d28891b29914aedf17f5e0333088545  ...        1

[5 rows x 8 columns]


In [ ]:
trans2 = pd.read_csv('/content/drive/My Drive/sample.csv')

In [ ]:
trans2.head()

,Unnamed: 0,chq_id,plant,chq_date,chq_position,client_id,material,sales_count,sales_sum,is_promo
0,26124569,01fb5fa1a778790afd08e8d7ef38e237,cccc8bc3d6630bf84a08f9ba361baea0,2017-08-03,25,6da30a50f81238d9b3e426c6b885d2ec,44ce960aed7d4c21001db08667a95bb0,2.00,23.58,0
1,25465983,867db9be630079c386bf8a5623482c9f,65b6f89e9041a2a78554f1c3db6c13e9,2017-07-22,44,2494649ed77f876d8bb818e41e65a705,184790d7f3250d7da915e71cf32ad255,1.00,149.99,0
2,14300781,27dbe224d498e0675aaaf0f0e198926a,2c393e45170ca8e02cc1b670e799da1b,2017-04-20,35,6934eef0d62f7e451fe803543c0d58a7,92c77ef0ad96da2bcf81e5f8d854e40b,4.00,159.56,1
3,22749195,4f90e9a0ab96ac01804a29e717ddadc8,9423f9da33f4c25382bf9dd96812916d,2017-06-19,9,f7d99bf6d08b1d733c4a38caa7283ef7,c2f210cc3ba0ea02c319eedd17d70a69,1.59,111.28,0
4,24955029,6bbdab43883e74f727ace712ee586017,66e78b22162c1774129ea149e0171b52,2016-10-09,4,a61459a5239c680fdbabe968df231ef4,ded0f55a75324cefcc53249559331e18,1.00,69.99,1


In [ ]:
trans2_grouped = trans2.groupby('client_id')

In [ ]:
clients_dict = {}

for trans_by_client_id in tqdm(trans2_grouped):
  client_id = trans_by_client_id[0]
  clients_dict[client_id] = {}
  trans_by_client_chq = trans_by_client_id[1].groupby('chq_id')
  
  for client_chq in trans_by_client_chq:
    clients_dict[client_id][client_chq[0]] = {}
    for column in ['plant', 'chq_date', 'material', 'sales_count', 'sales_sum', 'is_promo']:
      clients_dict[client_id][client_chq[0]][column] = client_chq[1].iloc[0][column]


  1%|▏         | 1299/91423 [00:22<21:30, 69.84it/s]

KeyboardInterrupt: ignored

In [ ]:
print(clients_dict.keys())

dict_keys(['0000d48a8891cd462fdf01ea861ec53e', '00028a1dcb30863404a386961a69300e', '0004274683494a9eca5e70ef97299d55', '00072f81efcb3481861f7300c5e2a88a', '00076596eb749cf10c2e984621dffe95', '00076dd9aa78642b7254fef37a1cdb3e', '0008322584b6781d656d7299e80c47f8', '00084b0b8d2649ed19e7a4df5062349b', '0008755f60606b6f8f8ea75ab98a9570', '0008cb44b4f40b04132d6dad44ea0d2c', '0008d7ebeb88886c8da5df07ac2d075f', '0009dae8ad7e1989054a19df18375134', '000a16b7265576e37c076f1423fa45fb', '000ac33e8990a55ba1ddd33b6300e4d3', '000b3314b6b5b91bc91a13ee9a87da25', '000bd6b9beffa9a3fca11224fe36cf2d', '000dab8b2c08ebd7f60d3782d9a4a500', '000df145bc1b56b3c028932d6173e9e6', '000e77c27e3910bdb783165635a05fdd', '000eb74fd036f9bd8b87a1d94e8b9e1b', '000f4f8196b582e33bd0aa594b59fb53', '000ff71a020b2e94076c4883341fc3dd', '001129b9d9e02f30510f6596033d2b99', '0012e4f89351ccba136aa25bfa7a9b18', '00135062b7f1c95750a3da3b5f788283', '0013ea6bf0c02f52cc4de45b8431f47e', '0013f409ab5a0a37aa999e6dc335710e', '001446e6bb2cc2e5

In [ ]:
print(0000d48a8891cd462fdf01ea861ec53e.keys())

dict_keys(['0000d48a8891cd462fdf01ea861ec53e', '00028a1dcb30863404a386961a69300e', '0004274683494a9eca5e70ef97299d55', '00072f81efcb3481861f7300c5e2a88a', '00076596eb749cf10c2e984621dffe95', '00076dd9aa78642b7254fef37a1cdb3e', '0008322584b6781d656d7299e80c47f8', '00084b0b8d2649ed19e7a4df5062349b', '0008755f60606b6f8f8ea75ab98a9570', '0008cb44b4f40b04132d6dad44ea0d2c', '0008d7ebeb88886c8da5df07ac2d075f', '0009dae8ad7e1989054a19df18375134', '000a16b7265576e37c076f1423fa45fb', '000ac33e8990a55ba1ddd33b6300e4d3', '000b3314b6b5b91bc91a13ee9a87da25', '000bd6b9beffa9a3fca11224fe36cf2d', '000dab8b2c08ebd7f60d3782d9a4a500', '000df145bc1b56b3c028932d6173e9e6', '000e77c27e3910bdb783165635a05fdd', '000eb74fd036f9bd8b87a1d94e8b9e1b', '000f4f8196b582e33bd0aa594b59fb53', '000ff71a020b2e94076c4883341fc3dd', '001129b9d9e02f30510f6596033d2b99', '0012e4f89351ccba136aa25bfa7a9b18', '00135062b7f1c95750a3da3b5f788283', '0013ea6bf0c02f52cc4de45b8431f47e', '0013f409ab5a0a37aa999e6dc335710e', '001446e6bb2cc2e5

In [ ]:
clients_dict['0000d48a8891cd462fdf01ea861ec53e']

{'272d955e9fa15e17c0f9089bdf6283cd': {'chq_date': '2017-08-31',
  'is_promo': 0,
  'material': 'c5b4dfc37695199f02ec0218175e9c23',
  'plant': '6858fb45a3d3aef7c29322d3b68dffd1',
  'sales_count': 2.0,
  'sales_sum': 27.98},
 '2977068f5533ac900f577fc571e9464b': {'chq_date': '2017-06-21',
  'is_promo': 0,
  'material': 'da163f4dfbea1578d9b64616e35bf66d',
  'plant': '7f8bb0fe8b33780a08fe6b60ced14529',
  'sales_count': 1.0,
  'sales_sum': 76.09},
 '2a4f1975d20e88b7a49a121a89d2a109': {'chq_date': '2016-12-13',
  'is_promo': 1,
  'material': '24122642f6b2b34296825cff981cf959',
  'plant': '6858fb45a3d3aef7c29322d3b68dffd1',
  'sales_count': 1.0,
  'sales_sum': 49.99},
 '36fc5b66638cd6882db5484148855aad': {'chq_date': '2017-08-11',
  'is_promo': 0,
  'material': '1907a86ec2794b04a0edfd1875e5ed0c',
  'plant': '6858fb45a3d3aef7c29322d3b68dffd1',
  'sales_count': 1.0,
  'sales_sum': 134.99},
 '436c322f5ec688c68c87b47aebee8d9c': {'chq_date': '2017-07-20',
  'is_promo': 1,
  'material': 'cc03c49ceb8

In [ ]:
with open('trans.pickle', 'wb') as handle:
    pickle.dump(clients_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('trans.pickle', 'rb') as handle:
    b = pickle.load(handle)

In [ ]:
b.keys()

dict_keys(['0000d48a8891cd462fdf01ea861ec53e', '00028a1dcb30863404a386961a69300e', '0004274683494a9eca5e70ef97299d55', '00072f81efcb3481861f7300c5e2a88a', '00076596eb749cf10c2e984621dffe95', '00076dd9aa78642b7254fef37a1cdb3e', '0008322584b6781d656d7299e80c47f8', '00084b0b8d2649ed19e7a4df5062349b', '0008755f60606b6f8f8ea75ab98a9570', '0008cb44b4f40b04132d6dad44ea0d2c', '0008d7ebeb88886c8da5df07ac2d075f', '0009dae8ad7e1989054a19df18375134', '000a16b7265576e37c076f1423fa45fb', '000ac33e8990a55ba1ddd33b6300e4d3', '000b3314b6b5b91bc91a13ee9a87da25', '000bd6b9beffa9a3fca11224fe36cf2d', '000dab8b2c08ebd7f60d3782d9a4a500', '000df145bc1b56b3c028932d6173e9e6', '000e77c27e3910bdb783165635a05fdd', '000eb74fd036f9bd8b87a1d94e8b9e1b', '000f4f8196b582e33bd0aa594b59fb53', '000ff71a020b2e94076c4883341fc3dd', '001129b9d9e02f30510f6596033d2b99', '0012e4f89351ccba136aa25bfa7a9b18', '00135062b7f1c95750a3da3b5f788283', '0013ea6bf0c02f52cc4de45b8431f47e', '0013f409ab5a0a37aa999e6dc335710e', '001446e6bb2cc2e5

In [ ]:
b['0000d48a8891cd462fdf01ea861ec53e']

{'272d955e9fa15e17c0f9089bdf6283cd': {'plant': '6858fb45a3d3aef7c29322d3b68dffd1'},
 '2977068f5533ac900f577fc571e9464b': {'plant': '7f8bb0fe8b33780a08fe6b60ced14529'},
 '2a4f1975d20e88b7a49a121a89d2a109': {'plant': '6858fb45a3d3aef7c29322d3b68dffd1'},
 '36fc5b66638cd6882db5484148855aad': {'plant': '6858fb45a3d3aef7c29322d3b68dffd1'},
 '436c322f5ec688c68c87b47aebee8d9c': {'plant': '6858fb45a3d3aef7c29322d3b68dffd1'},
 '50b662f8dc3d49588b12a9a3c529cf74': {'plant': '7f8bb0fe8b33780a08fe6b60ced14529'},
 '5cad89bca4f84f15a0964bb801772c35': {'plant': '7f8bb0fe8b33780a08fe6b60ced14529'},
 '5e58f7ad4e48291073ee4807d952dd62': {'plant': '6858fb45a3d3aef7c29322d3b68dffd1'},
 '5f3dffc33e579cfce95c989e7a2853d3': {'plant': '6858fb45a3d3aef7c29322d3b68dffd1'},
 '7fe8aeee4114a557a42c604c23132dd0': {'plant': '6858fb45a3d3aef7c29322d3b68dffd1'},
 '8d9b38d9a8fb90def7fbf8cca2d8d83b': {'plant': '7f8bb0fe8b33780a08fe6b60ced14529'},
 '905d1530fb707387e46ca3e03f81f3c9': {'plant': '6858fb45a3d3aef7c29322d3b68d